**классификатор текстов**

SnowballStemmer+TfidfVectorizer

_Евгений Борисов <esborisov@sevsu.ru>_

## тексты

In [1]:
import pandas as pd
pd.options.display.precision = 2 
pd.options.display.max_colwidth = 200 

from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
# загружаем тексты
data = pd.read_pickle('../data/news.pkl.gz')
print('записей:',len(data))
data.sample(2)

записей: 3196


,text,tag
2777,Специалисты Apple планируют начать широкомасштабную кампанию по борьбе со спамом в стандартном для «яблочных» гаджетов приложении «Календарь» из macOS и iOS. В последнее время пользователи активно...,tech
1891,"В Париже найдена неизвестная работа Леонардо да Винчи. Стоимость работы оценивается в €15 млн, передает издание Telegraph.\n\nСообщается, что в марте текущего года французский доктор на пенсии, по...",culture


## токенайзер со стемингом и очисткой

In [3]:
# from nltk.tokenize import sent_tokenize as nltk_sentence_split
from nltk.tokenize import word_tokenize as nltk_tokenize_word
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords as nltk_stopwords
import re

stemmer = SnowballStemmer('russian')
stopwords = set(nltk_stopwords.words('russian'))



def tokenizer(text,stemmer=stemmer,stopwords=stopwords):
    return [
            stemmer.stem(t) # выполняем стеминг
            for t in nltk_tokenize_word( # разбиваем текст на слова
                re.sub(r'</?[a-z]+>',' ',text), # удаляем xml tag 
                language='russian'
            ) 
            if not (
               (len(t)<3) # выкидываем очень короткие слова
               or re.match(r'^[^a-zA-ZЁёА-я]+$', t) # выкидываем токены не содержащие букв
               or re.match(r'^(\w)\1+$', t)  # выкидываем токены из одного повторяющегося символа
               or re.match(r'^[^a-zA-ZЁёА-я].*$', t)  # выкидываем токены начинающиеся не с буквы
               or (t in stopwords) # выкидываем предлоги, союзы и т.п.    
            )
        ] 
    
# data['text'].progress_apply(tokenizer)

## выполняем частотный анализ

In [4]:
%%time

from sklearn.feature_extraction.text import TfidfVectorizer

# использования токенайзера вместе с векторайзером
tf_model = TfidfVectorizer(
        use_idf=False, # не используем обратную частоту
        norm='l2', # нормируем TF
        tokenizer=tokenizer, # ф-ция токенайзер
        token_pattern=None, # отключаем дефолтный токенайзер
    )


data_tf = tf_model.fit_transform( data['text'] )

CPU times: user 20.2 s, sys: 12.6 ms, total: 20.2 s
Wall time: 20.2 s


In [5]:
data_tf.shape

(3196, 37571)

In [6]:
vcb1 = sorted(tf_model.vocabulary_)
print(len(vcb1))
pd.Series(vcb1).sample(30)

37571


30251        ски-пасс
8163      выздоровлен
7399             водн
11157             дтп
10062         декольт
24019          по-ком
4107          алогичн
7422     водохранилищ
9349            госсм
16172    кузбассэнерг
21303     общегородск
6821         вертолет
14685      кинохроник
16248          кунг-ф
36487      шкерманков
34347          уточек
36582         шоу-кар
26774        произошл
14454         квантов
7162          виталик
32210            сырн
11521             еэк
2991            tiggo
31269          спирит
14290       каролинск
34173         уродств
2867     stradivarius
36984      электричек
17808             мая
27528         радикал
dtype: object

## формируем датасеты

In [7]:
labels = { t:i for i,t in enumerate(sorted(set(data['tag']))) }
labels

{'auto': 0,
 'culture': 1,
 'economics': 2,
 'health': 3,
 'incident': 4,
 'politics': 5,
 'realty': 6,
 'reclama': 7,
 'science': 8,
 'social': 9,
 'sport': 10,
 'tech': 11,
 'woman': 12}

In [8]:
y = data['tag'].map(labels).values
y

array([5, 1, 1, ..., 8, 5, 9])

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( data_tf, y, test_size=0.3, random_state=326 )
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2237, 37571), (2237,), (959, 37571), (959,))

## обучаем классификатор

In [10]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='hinge',max_iter=1000, tol=0.19)
clf.fit(X_train,y_train)

SGDClassifier(tol=0.19)

## тестируем

In [11]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [12]:
# для правильный ответов на учебном наборе
o = clf.predict(X_train)
accuracy_score(y_train,o)

0.9995529727313366

In [13]:
# для правильный ответов на тестовом наборе
o = clf.predict(X_test)
accuracy_score(y_test,o)

0.8706986444212722

In [14]:
print( classification_report(y_test,o) )

              precision    recall  f1-score   support

           0       0.95      0.84      0.89        87
           1       0.81      0.94      0.87       101
           2       0.83      0.83      0.83        76
           3       0.81      0.71      0.76        31
           4       0.88      0.91      0.90       123
           5       0.87      0.94      0.90       177
           6       0.94      0.76      0.84        21
           7       0.90      0.53      0.67        17
           8       0.88      1.00      0.94        68
           9       0.59      0.36      0.45        44
          10       0.97      0.99      0.98       118
          11       0.82      0.81      0.81        89
          12       1.00      0.86      0.92         7

    accuracy                           0.87       959
   macro avg       0.87      0.81      0.83       959
weighted avg       0.87      0.87      0.87       959



---

In [26]:
pd.DataFrame( 
        confusion_matrix(y_test,o), 
        columns=sorted(labels.keys()), 
        index=sorted(labels.keys()),
    ).style.background_gradient(cmap='Blues')

,auto,culture,economics,health,incident,politics,realty,reclama,science,social,sport,tech,woman
auto,73,0,1,0,10,1,0,0,0,0,0,2,0
culture,0,95,0,0,0,3,0,0,1,1,0,1,0
economics,0,1,63,0,0,8,0,0,0,3,0,1,0
health,0,5,0,22,1,1,0,0,1,1,0,0,0
incident,2,1,1,0,112,3,0,0,0,1,0,3,0
politics,0,2,3,0,0,166,0,0,0,2,1,3,0
realty,0,1,1,1,0,0,16,1,0,1,0,0,0
reclama,0,2,1,0,0,0,0,9,0,1,2,2,0
science,0,0,0,0,0,0,0,0,68,0,0,0,0
social,1,8,3,2,2,5,1,0,2,16,0,4,0


---

In [27]:
data['predict'] = pd.Series( clf.predict(data_tf) ).map( { labels[k]:k for k in labels } )

In [28]:
data

,text,tag,predict
0,"В Саудовской Аравии сняли первый антитеррористический мультфильм -\nтрехминутную ленту ""Внимание!"". ""Внимание!"" отражает точку зрения мирового\nсообщества на войну, развязанную терроризмом, и поэт...",politics,culture
1,"Вчера вечером в Японии состоялась премьера голливудского фильма о гейшах,\nвызвавшая негодование в связи с тем, что эти девушки представлены\nпроститутками, а играющие их актрисы - китаянки. Мало ...",culture,culture
2,"Российский кинорежиссер и генеральный директор киноконцерна ""Мосфильм""\nКарен Шахназаров награжден ""Золотой пирамидой"" на XXIX Каирском кинофестивале\nза выдающийся вклад в мировое киноискусство. ...",culture,culture
3,30 ноября выдающейся российской балерине Майе Плисецкой будет вручена\nмедаль имени княгини Барборы Радвилайте. Церемония награждения состоится\nв Вильнюсе в Литовском национальном театре оперы и ...,culture,culture
4,"Гарольд Пинтер не приедет за Нобелевской премией из-за болезниАнглийский\nдраматург Гарольд Пинтер, получивший Нобелевскую премию по литературе в\n2005 году, отправит в Стокгольм видеозапись своей...",culture,culture
...,...,...,...
3191,Православие.Ру В сентябре 2010 года Святейший Патриарх Кирилл посетит Камчатку\nВопросы подготовки к поездке Предстоятеля Русской Православной Церкви на\nКамчатку обсуждались на встрече Святейшего...,social,social
3192,Интерфакс Религия (interfax-religion.ru) В Минрегионразвития призывают\nроссиян не бояться своего духовного наследия подобно Европе В министерстве\nрегионального развития РФ считают крайне важным ...,social,culture
3193,"Окно возможностей В Эвенкинском муниципальном районе приступили к\nисследованию традиционного уклада жизни. В Эвенкию прибыла экспедиция под\nруководством профессора, заведующего кафедрой менеджме...",science,science
3194,"ИТАР-ТАСС. Новости из властных структур. Совет Федерации предлагает определить\nособенности традиционной охоты коренных народов Севера, Сибири и Дальнего\nВостока Совет Федерации внес на рассмотр...",politics,politics


In [33]:
# idx = pd.IndexSlice
# data_ = data.sample(50)
# slice_ = idx[idx[ data_['tag']!=data_['predict'] ], :]
# data_.style.set_properties(**{'background-color': '#ffffb3'}, subset=slice_)

In [ ]:
# Введение в анализ текстовой информации с помощью Python и методов машинного обучения
# https://habr.com/ru/post/205360/